In [ ]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import subprocess
import numpy as np

from joblib import Parallel, delayed
import shutil

In [ ]:
## Enter sudo password here
password = "3108"

In [ ]:

def str2datetime(s):
    parts = s.split('.')
    dt = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S")
    return dt.replace(microsecond=int(parts[1]))


def remove_outliers(x, outlierConstant = 1.5):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    removedList = []
    for y in a.tolist():
        if y >= quartileSet[0] and y <= quartileSet[1]:
            resultList.append(y)
        else:
            removedList.append(y)
    return (resultList, removedList)
def getThroughput(experiment):
    
    temp_lines = []

    result = subprocess.check_output('cat ../'+experiment+'/nohup_client_*', shell=True)


    result = result.decode("utf-8") 

    lines = result.split('\n')

    commit_pat = re.compile('([^[].*) \[hotstuff info\] ([0-9.]*)$')
#     commit_pat = re.compile('([^[].*) \[hotstuff info\] got*')

    Times = []
    lats = []

    for line in lines:
        m = commit_pat.match(line)
        if m:
            try:
                temp_lines.append(line)
                time_item = float((line.split(' ')[1]).split(':')[0])*3600 + float((line.split(' ')[1]).split(':')[1])*60 + \
                float((line.split(' ')[1]).split(':')[2])
                Times.append(time_item)        

                lats.append(float(m.group(2)))
            except:
                continue


    begin_time = None
    values = []
    interval = 1
    cnt = 0
    Times.sort()
    for timestamp in Times:
        if begin_time is None:
            begin_time = timestamp
            next_begin_time = timestamp +interval

        while timestamp >= next_begin_time:
            begin_time = next_begin_time
            next_begin_time += interval
            values.append(cnt)
            cnt = 0
        cnt += 1
    values.append(cnt)

#     lats, _ = remove_outliers(lats)
#     print(len(values))


    return np.arange(len(values))*interval,np.array(values)/interval, lats, temp_lines


In [ ]:
with open('all_local_ips','r') as firstfile, open('all_external_ips','w') as secondfile:
      
    # read content from first file
    for line in firstfile:

        secondfile.write(line)
        
        
with open('all_local_ips','r') as firstfile, open('all_internal_ips','w') as secondfile:
      
    # read content from first file
    for line in firstfile:

        secondfile.write(line)

In [ ]:
nclients = 2
nnodes = 8

In [ ]:
f = open('all_external_ips', "r")
data1 = [line.strip() for line in f.readlines()]

In [ ]:
f = open('all_internal_ips', "r")
data2 = [line.strip() for line in f.readlines()]

In [ ]:
data2, len(data2)

In [ ]:
data = []

In [ ]:

skip_instances = 0
    
for i in range(skip_instances, len(data1)):
    if data1[i]!='None':
        data.append(data1[i] + '    ' + data2[i])


In [ ]:
data, len(data)

In [ ]:
with open('replicas.txt', 'w') as f:
    for line in data[:nnodes]:
        f.write("%s\n" % line)

In [ ]:
data, len(data)

In [ ]:
with open('clients.txt', 'w') as f:
    for line in data[nnodes:]:

        f.write("127.0.0.1\n")        

In [ ]:
os.system('cat clients.txt')

In [ ]:

with open('all_external_ips') as f:
    lines = f.read().splitlines()


lines = [x for x in lines if x!='None']
lines


In [ ]:
lines, len(lines)

In [ ]:


def run_server_node(i):
    ip = lines[i]
    if ip!='None':

        current = (ip.split('.'))
        command = 'ssh  -o StrictHostKeyChecking=no -i /home/tejas/Downloads/MAKWest.pem '+'ubuntu@ec2-'+str(current[0])+'-'\
              +str(current[1])+'-'+str(current[2])+'-'+str(current[3])+'.us-west-1.compute.amazonaws.com -T "cd /home/ubuntu/testbed/'+experiment+'/0/conf/; sudo nohup ../../../../libhotstuff/examples/hotstuff-app --conf '+'/home/ubuntu/testbed/'+experiment+'/0/conf/'+'hotstuff.gen-sec'+str(i)+'.conf >../nohup.out 2>&1 &"'

        print(command)
        os.system(command)
        
        
        

In [ ]:
experiment = '../temp'

In [ ]:
os.system(f'cd ../temp; echo {password} | sudo -S  rm *')

In [ ]:
node_list = list(range(len(lines)))
node_list

node_list_server = node_list[:nnodes]
node_list_client = node_list[-nclients:]

node_list_server, node_list_client


In [ ]:
nnodes

In [ ]:
os.system(f'echo {password} | sudo -S rm ../'+experiment+'/nohup_*')


os.system(f'echo {password} | sudo -S killall hotstuff-app')
os.system(f'echo {password} | sudo -S killall hotstuff-client')
os.system(f'echo {password} | sudo -S killall nohup')


In [ ]:
os.system(f'echo {password} | sudo -S sh job_mini.sh '+ experiment)

In [ ]:
        
def run_node(i):
    ip = lines[i]
#     if ip!='None' and( ip in nodeIps[:-3]):
    if ip!='None' and ip in lines[:nnodes]:

        current = (ip.split('.'))
        command = f'echo {password} | sudo -S nohup ../../examples/hotstuff-app --conf hotstuff.gen-sec'+str(i)+'.conf > ../temp/nohup_'+str(i)+'.out 2>&1 &'

        print(command)
        os.system(command)
        
        





results = Parallel(n_jobs=len(lines))(delayed(run_node)(i) for i in [0,1,2,3,4,5,6,7])

print(results)


time.sleep(10)

In [ ]:
time.sleep(10)

In [ ]:
def run_client(i):
    ip = lines[i]
    client_no = i - nnodes
    if ip!='None' and( ip in lines[nnodes:]):
        
        current = (ip.split('.'))
        command = f'echo {password} | sudo -S nohup ../../examples/hotstuff-client --idx '+str(client_no)+' --iter -1 --max-async 4 > ../temp/nohup_client_'+str(i)+'.out 2>&1 &'
        
        print(command)
        os.system(command)



results = Parallel(n_jobs=len(lines))(delayed(run_client)(i) for i in [8,9])

# time.sleep(20)

In [ ]:
# def run_client(i):
#     ip = lines[i]
#     client_no = i - nnodes
#     if ip!='None' and( ip in lines[nnodes:]):
        
#         current = (ip.split('.'))
#         command = f'echo {password} | sudo -S nohup ../../examples/hotstuff-client --idx '+str(client_no)+' --reconfig_client 1 --iter -1 --max-async 4 > ../temp/nohup_client_'+str(i)+'.out 2>&1 &'
        
#         print(command)
#         os.system(command)

# results = Parallel(n_jobs=len(lines))(delayed(run_client)(i) for i in [10])

In [ ]:
time.sleep(30)

In [ ]:
# time.sleep(10)
# os.system(f'echo {password} | sudo -S killall hotstuff-client')

# def run_client(i):
#     ip = lines[i]
#     client_no = i - nnodes
#     if ip!='None' and( ip in lines[nnodes:]):
        
#         current = (ip.split('.'))
#         command = f'echo {password} | sudo -S nohup ../../examples/hotstuff-client --idx '+str(client_no)+' --iter -1 --max-async 4 > ../temp/nohup_client_'+str(i+2)+'.out 2>&1 &'
        
#         print(command)
#         os.system(command)



# results = Parallel(n_jobs=len(lines))(delayed(run_client)(i) for i in [8,9])
# time.sleep(50)



In [ ]:
os.system(f'echo {password} | sudo -S killall hotstuff-client')


os.system(f'echo {password} | sudo -S killall hotstuff-app')
os.system(f'echo {password} | sudo -S killall nohup')

In [ ]:
experiment = 'temp'
X,Y, lats, temp = getThroughput(experiment)
print(np.sum(Y)/len(Y), np.average(lats))

In [ ]:
fig = pl.figure(figsize = (12, 8))
fig.patch.set_facecolor('white')
pl.plot(X[:-1],Y[:-1], '-*')
pl.xlabel('Time')
pl.ylabel('Average Throughput (txn/sec)')
pl.show()
pl.clf()


fig = pl.figure(figsize = (12, 8))
fig.patch.set_facecolor('white')
# pl.plot(X[:-1],Y[:-1], '-*')
pl.plot(X[:-1],np.cumsum(Y[:-1]), '-*')

pl.xlabel('Time')
pl.ylabel('Cumulative Throughput (txns)')
pl.show()
pl.clf()